## Welcome to the Second Lab - Week 1, Day 3

Today we will work with lots of models! This is a way to get comfortable with APIs.

<table style="margin: 0; text-align: left; width:100%">
    <tr>
        <td style="width: 150px; height: 150px; vertical-align: middle;">
            <img src="../assets/stop.png" width="150" height="150" style="display: block;" />
        </td>
        <td>
            <h2 style="color:#ff7800;">Important point - please read</h2>
            <span style="color:#ff7800;">The way I collaborate with you may be different to other courses you've taken. I prefer not to type code while you watch. Rather, I execute Jupyter Labs, like this, and give you an intuition for what's going on. My suggestion is that you carefully execute this yourself, <b>after</b> watching the lecture. Add print statements to understand what's going on, and then come up with your own variations.<br/><br/>If you have time, I'd love it if you submit a PR for changes in the community_contributions folder - instructions in the resources. Also, if you have a Github account, use this to showcase your variations. Not only is this essential practice, but it demonstrates your skills to others, including perhaps future clients or employers...
            </span>
        </td>
    </tr>
</table>

In [3]:
# Start with imports - ask ChatGPT to explain any package that you don't know

import os
import json
from dotenv import load_dotenv
from openai import OpenAI
#from anthropic import Anthropic
from IPython.display import Markdown, display

In [4]:
# Always remember to do this!
load_dotenv(override=True)

True

In [5]:
# Print the key prefixes to help with any debugging

openai_api_key = os.getenv('OPENAI_API_KEY')
# anthropic_api_key = os.getenv('ANTHROPIC_API_KEY')
google_api_key = os.getenv('GOOGLE_API_KEY')
deepseek_api_key = os.getenv('DEEPSEEK_API_KEY')
groq_api_key = os.getenv('GROQ_API_KEY')

if openai_api_key:
    print(f"OpenAI API Key exists and begins {openai_api_key[:8]}")
else:
    print("OpenAI API Key not set")
    
# if anthropic_api_key:
#     print(f"Anthropic API Key exists and begins {anthropic_api_key[:7]}")
# else:
#     print("Anthropic API Key not set (and this is optional)")

if google_api_key:
    print(f"Google API Key exists and begins {google_api_key[:2]}")
else:
    print("Google API Key not set (and this is optional)")

if deepseek_api_key:
    print(f"DeepSeek API Key exists and begins {deepseek_api_key[:3]}")
else:
    print("DeepSeek API Key not set (and this is optional)")

if groq_api_key:
    print(f"Groq API Key exists and begins {groq_api_key[:4]}")
else:
    print("Groq API Key not set (and this is optional)")

OpenAI API Key exists and begins sk-proj-
Google API Key exists and begins AI
DeepSeek API Key exists and begins sk-
Groq API Key exists and begins gsk_


In [6]:
request = "Please come up with a challenging, nuanced question that I can ask a number of LLMs to evaluate their intelligence. "
request += "Answer only with the question, no explanation."
messages = [{"role": "user", "content": request}]

In [7]:
messages

[{'role': 'user',
  'content': 'Please come up with a challenging, nuanced question that I can ask a number of LLMs to evaluate their intelligence. Answer only with the question, no explanation.'}]

In [8]:
openai = OpenAI()
response = openai.chat.completions.create(
    model="gpt-4o-mini",
    messages=messages,
)
question = response.choices[0].message.content
print(question)


How would you design an educational curriculum that effectively balances the development of critical thinking skills with the mastery of specific knowledge in a rapidly changing technological landscape?


In [9]:
competitors = []
answers = []
messages = [{"role": "user", "content": question}]

In [10]:
# The API we know well

model_name = "gpt-4o-mini"

response = openai.chat.completions.create(model=model_name, messages=messages)
answer = response.choices[0].message.content

display(Markdown(answer))
competitors.append(model_name)
answers.append(answer)

Designing an educational curriculum that balances critical thinking skills and mastery of specific knowledge, especially in a rapidly changing technological landscape, requires a multifaceted approach. Here’s a strategic framework that educators could consider implementing:

### 1. **Curriculum Framework:**

#### A. **Integrated Learning Objectives:**
   - **Critical Thinking Skills:** Identify specific competencies like problem-solving, analytical reasoning, creative thinking, and decision-making.
   - **Content Mastery:** Establish core knowledge areas relevant to the technological landscape (e.g., coding fundamentals, data literacy, digital ethics).

#### B. **Interdisciplinary Approach:**
   - Encourage connections between subjects (STEM, humanities, arts) to demonstrate how critical thinking applies across domains.
   - Incorporate real-world problems and case studies that require cross-disciplinary knowledge and critical analysis.

### 2. **Teaching Methodologies:**

#### A. **Project-Based Learning (PBL):**
   - Design projects that require students to apply specific knowledge and utilize critical thinking to develop solutions.
   - Encourage collaboration and peer review to foster discussion and diverse perspectives.

#### B. **Socio-Emotional Learning:**
   - Integrate social-emotional skills to help students navigate ethical dilemmas in tech, enhancing their decision-making capabilities.

#### C. **Flipped Classroom Model:**
   - Assign foundational knowledge acquisition through online modules, allowing class time for discussion, debates, and critical analysis.

### 3. **Assessment Practices:**

#### A. **Formative Assessment:**
   - Use ongoing assessments to provide feedback on student understanding and critical thinking applications, rather than relying solely on high-stakes testing.

#### B. **Performance-Based Assessments:**
   - Include assessments that require demonstration of critical thinking, such as debates, presentations, or portfolio projects showcasing problem-solving processes.

### 4. **Curriculum Content:**

#### A. **Future-Focused Themes:**
   - Explore themes such as ethical implications of AI, data privacy, and environmental impacts of technology to ground learning in current events.
   - Incorporate discussions about the societal impacts of technology, preparing students to think critically about the tools they create and use.

#### B. **Career Readiness:**
   - Integrate skills for future careers, including coding, data analysis, and digital media literacy with a focus on collaboration and innovation.

### 5. **Teacher Training and Support:**

#### A. **Professional Development:**
   - Provide ongoing training for educators on fostering critical thinking and integrating technology effectively into their curricula.
   - Share best practices and collaborative teaching strategies among educators to promote a culture of innovation.

#### B. **Interdisciplinary Collaboration:**
   - Encourage regular meetings between educators of different subjects to design cohesive lessons and assess student progress holistically.

### 6. **Community and Industry Engagement:**

#### A. **Partnerships:**
   - Collaborate with local businesses and tech companies to provide students with real-world experiences and mentorship opportunities.
   - Organize workshops or guest lectures by industry experts to illustrate the application of critical thinking and knowledge in professional contexts.

### 7. **Iterative Feedback and Adjustment:**

#### A. **Curriculum Review:**
   - Develop mechanisms for regularly reviewing and updating the curriculum based on technological advances and educational research.
   - Gather feedback from students, educators, and industry stakeholders to continually refine and improve the curriculum.

By ensuring a balance between critical thinking and content mastery, this curriculum framework prepares students not just to navigate the current technological landscape, but to be adaptable, innovative, and responsible citizens in the future.

In [11]:
# Anthropic has a slightly different API, and Max Tokens is required

model_name = "claude-3-7-sonnet-latest"

claude = Anthropic()
response = claude.messages.create(model=model_name, messages=messages, max_tokens=1000)
answer = response.content[0].text

display(Markdown(answer))
competitors.append(model_name)
answers.append(answer)

NameError: name 'Anthropic' is not defined

In [12]:
gemini = OpenAI(api_key=google_api_key, base_url="https://generativelanguage.googleapis.com/v1beta/openai/")
model_name = "gemini-2.0-flash"

response = gemini.chat.completions.create(model=model_name, messages=messages)
answer = response.choices[0].message.content

display(Markdown(answer))
competitors.append(model_name)
answers.append(answer)

Designing an educational curriculum that balances critical thinking with specific knowledge in a rapidly changing technological landscape requires a multifaceted approach. Here's a breakdown of the key elements:

**I. Foundational Principles:**

*   **Lifelong Learning Mindset:** The primary goal is to equip students with the tools and mindset to be lifelong learners, capable of adapting to new technologies and information throughout their lives.
*   **Transferable Skills:** Focus on skills that are applicable across different contexts and technologies, rather than solely on platform-specific expertise.
*   **Problem-Based & Inquiry-Based Learning:**  Use real-world problems and questions to drive learning, encouraging students to actively seek knowledge and solutions.
*   **Interdisciplinary Approach:** Integrate different subjects and disciplines to foster a holistic understanding of complex issues.
*   **Ethical Considerations:** Emphasize the ethical implications of technology and its impact on society.
*   **Personalized Learning:** Cater to individual learning styles, paces, and interests to maximize engagement and effectiveness.

**II. Core Components of the Curriculum:**

**A.  Critical Thinking Skills Development:**

*   **Logic and Reasoning:**
    *   Formal and informal logic.
    *   Argumentation and debate.
    *   Identifying fallacies and biases.
    *   Evidence-based reasoning.
*   **Information Literacy:**
    *   Source evaluation (credibility, bias, accuracy).
    *   Effective search strategies.
    *   Synthesizing information from multiple sources.
    *   Understanding the difference between fact, opinion, and interpretation.
*   **Problem-Solving:**
    *   Design thinking.
    *   Systems thinking.
    *   Root cause analysis.
    *   Creative problem-solving techniques (brainstorming, mind mapping).
*   **Analytical Thinking:**
    *   Data analysis and interpretation.
    *   Statistical reasoning.
    *   Identifying patterns and trends.
    *   Drawing inferences and making predictions.
*   **Decision-Making:**
    *   Developing decision criteria.
    *   Evaluating alternatives.
    *   Risk assessment.
    *   Ethical decision-making.

**B.  Specific Knowledge Domains (Technologically Relevant):**

*   **Core STEM Foundations:**
    *   **Mathematics:**  Algebra, calculus, statistics, discrete mathematics (foundational for many tech fields).
    *   **Science:**  Physics, chemistry, biology (depending on the technology focus).
    *   **Computer Science:**  Programming fundamentals, data structures, algorithms, software engineering principles.
    *   **Engineering:**  Principles of design, problem-solving, and implementation (electrical, mechanical, chemical, etc.).
*   **Emerging Technologies:**  Curriculum should dynamically adapt to include:
    *   **Artificial Intelligence (AI) & Machine Learning:** Fundamentals, ethical considerations, applications.
    *   **Data Science & Big Data:** Data analysis, visualization, database management.
    *   **Cybersecurity:**  Network security, cryptography, ethical hacking.
    *   **Cloud Computing:** Infrastructure as a Service (IaaS), Platform as a Service (PaaS), Software as a Service (SaaS).
    *   **Internet of Things (IoT):**  Sensors, data acquisition, network protocols.
    *   **Blockchain Technology:**  Decentralized systems, cryptocurrencies, smart contracts.
    *   **Biotechnology & Genetic Engineering:**  (Depending on the context)
    *   **Renewable Energy Technologies:** (Depending on the context)
*   **Digital Literacy:**
    *   Proficiency in using various software and hardware tools.
    *   Understanding of online communication and collaboration tools.
    *   Digital citizenship and ethical online behavior.

**C.  Integrating Critical Thinking and Knowledge:**

*   **Project-Based Learning (PBL):**
    *   Students work on complex, real-world projects that require them to apply both knowledge and critical thinking skills.
    *   Examples: Developing a mobile app to solve a local problem, designing a sustainable energy solution, analyzing a large dataset to identify trends.
*   **Case Studies:**
    *   Analyze real-world scenarios where technology has had a significant impact, requiring students to consider ethical, social, and economic implications.
*   **Simulations and Virtual Labs:**
    *   Provide hands-on experience with complex systems and allow students to experiment and learn from mistakes in a safe environment.
*   **Debates and Discussions:**
    *   Encourage students to engage in respectful debates about controversial topics related to technology, requiring them to research, analyze, and articulate their positions.
*   **Research and Inquiry:**
    *   Provide opportunities for students to conduct independent research on topics of interest, fostering critical thinking and information literacy.
*   **Creative Activities:**
    *   Encourage students to use technology in creative ways, such as creating digital art, composing music, or designing games.
*   **Coding and Computational Thinking:**
    *   Learning to code not only teaches specific programming skills but also develops logical thinking, problem-solving, and algorithmic thinking.

**III. Implementation Strategies:**

*   **Curriculum Design:**
    *   **Modular Design:**  Break down the curriculum into smaller, flexible modules that can be easily updated and adapted to changing technology.
    *   **Backward Design:** Start with desired outcomes and then design the curriculum to achieve those outcomes.
    *   **Regular Curriculum Review:**  Establish a process for regularly reviewing and updating the curriculum to ensure that it remains relevant and aligned with industry needs.
*   **Teaching Methods:**
    *   **Active Learning:**  Emphasize student engagement through activities such as group work, discussions, and presentations.
    *   **Flipped Classroom:**  Students learn content outside of class and use class time for active learning and problem-solving.
    *   **Personalized Learning:**  Use technology to provide personalized learning experiences that cater to individual student needs.
    *   **Mentorship:**  Connect students with industry professionals who can provide guidance and support.
*   **Assessment:**
    *   **Formative Assessment:**  Use frequent, low-stakes assessments to monitor student progress and provide feedback.
    *   **Summative Assessment:**  Use comprehensive assessments, such as projects and exams, to evaluate student learning at the end of a unit or course.
    *   **Portfolio Assessment:**  Students create a portfolio of their work that demonstrates their skills and knowledge.
    *   **Competency-Based Assessment:**  Assess students based on their ability to demonstrate specific skills and knowledge, rather than on traditional grades.
*   **Technology Integration:**
    *   **Learning Management Systems (LMS):**  Use LMS platforms to deliver content, track student progress, and facilitate communication.
    *   **Collaboration Tools:**  Use online collaboration tools to encourage students to work together on projects.
    *   **Virtual Reality (VR) and Augmented Reality (AR):**  Use VR and AR to create immersive learning experiences.
    *   **Educational Games:**  Use games to make learning more engaging and fun.
*   **Faculty Development:**
    *   Provide faculty with ongoing training and support in the use of technology and effective teaching methods.
    *   Encourage faculty to collaborate with industry professionals to stay up-to-date on the latest trends and technologies.
*   **Industry Partnerships:**
    *   Partner with local businesses and organizations to provide students with internships, mentorships, and real-world learning experiences.
    *   Invite industry professionals to guest lecture in classes and provide feedback on curriculum design.
*   **Infrastructure and Resources:**
    *   Ensure that students have access to the necessary technology, including computers, software, and internet access.
    *   Provide students with access to online learning resources, such as e-books, videos, and tutorials.

**IV. Continuous Improvement:**

*   **Data Collection and Analysis:**  Collect data on student learning outcomes, faculty effectiveness, and curriculum relevance.
*   **Feedback Mechanisms:**  Establish mechanisms for gathering feedback from students, faculty, and industry partners.
*   **Adaptive Curriculum:**  Use data and feedback to continuously improve the curriculum and teaching methods.

**V.  Example Curriculum Structure (Illustrative):**

**Year 1: Foundations**

*   **Semester 1:**  Introduction to Critical Thinking, Foundational Math, Introduction to Computer Science (Programming Basics), Digital Literacy
*   **Semester 2:**  Logic and Reasoning, Data Analysis,  Introduction to Engineering Principles, Ethics in Technology

**Year 2: Building Blocks**

*   **Semester 3:**  Algorithms & Data Structures, Information Literacy, Statistics, Project-Based Learning (Introduction)
*   **Semester 4:**  Software Engineering, Database Fundamentals,  Problem-Solving Techniques,  Interdisciplinary Project (e.g., Design a system for environmental monitoring using IoT)

**Year 3: Specialization & Application**

*   **Semester 5 & 6:**  Choose a specialization track (e.g., AI, Cybersecurity, Data Science).  Track-specific coursework. Advanced Critical Thinking Seminars (focus on specific issues related to the chosen specialization). Project-Based Learning (More complex, longer-term projects).

**Year 4: Capstone & Real-World Experience**

*   **Semester 7 & 8:**  Capstone Project (Independent research and development of a significant project).  Internship or Industry Placement.  Advanced Topics Seminars (exploring cutting-edge research). Portfolio Development.

**Key Considerations for a Rapidly Changing Landscape:**

*   **Flexibility and Adaptability:** The curriculum must be designed to be easily updated and adapted as new technologies emerge.
*   **Focus on Fundamentals:**  Ensure a strong foundation in the underlying principles of technology, rather than focusing solely on specific tools or platforms.
*   **Emphasis on Continuous Learning:**  Equip students with the skills and mindset to be lifelong learners who can adapt to new technologies throughout their careers.
*   **Strong Industry Partnerships:**  Collaborate with industry partners to ensure that the curriculum is relevant to the needs of the workforce.

By implementing these principles and components, an educational curriculum can effectively balance the development of critical thinking skills with the mastery of specific knowledge in a rapidly changing technological landscape, preparing students for success in the 21st century.


In [ ]:
deepseek = OpenAI(api_key=deepseek_api_key, base_url="https://api.deepseek.com/v1")
model_name = "deepseek-chat"

response = deepseek.chat.completions.create(model=model_name, messages=messages)
answer = response.choices[0].message.content

display(Markdown(answer))
competitors.append(model_name)
answers.append(answer)

APIStatusError: Error code: 402 - {'error': {'message': 'Insufficient Balance', 'type': 'unknown_error', 'param': None, 'code': 'invalid_request_error'}}

In [13]:
groq = OpenAI(api_key=groq_api_key, base_url="https://api.groq.com/openai/v1")
model_name = "llama-3.3-70b-versatile"

response = groq.chat.completions.create(model=model_name, messages=messages)
answer = response.choices[0].message.content

display(Markdown(answer))
competitors.append(model_name)
answers.append(answer)


Designing an educational curriculum that balances critical thinking skills with the mastery of specific knowledge in a rapidly changing technological landscape requires a flexible, adaptive, and interdisciplinary approach. Here's a comprehensive framework to achieve this balance:

**Curriculum Pillars:**

1. **Foundational Knowledge**: Develop a strong foundation in core subjects like mathematics, physics, computer science, and engineering.
2. **Critical Thinking and Problem-Solving**: Integrate critical thinking, problem-solving, and analytical skills throughout the curriculum, using real-world examples and case studies.
3. **Technological Literacy**: Foster an understanding of the latest technologies, including emerging trends and their potential applications.
4. **Interdisciplinary Connections**: Emphasize the interconnectedness of subjects and encourage students to explore relationships between technology, society, and the environment.
5. **Soft Skills and Collaboration**: Develop essential soft skills, such as communication, teamwork, and time management, through project-based learning and group work.

**Curriculum Structure:**

1. **Modular Format**: Organize the curriculum into modular units, each focused on a specific topic or skill, allowing for flexibility and adaptability.
2. **Spiral Learning**: Use a spiral learning approach, where concepts are introduced, revisited, and expanded upon throughout the curriculum, promoting deeper understanding and mastery.
3. **Blended Learning**: Incorporate a mix of traditional teaching methods, online resources, and hands-on activities to cater to different learning styles and preferences.
4. **Project-Based Learning**: Emphasize project-based learning, where students work on real-world problems, fostering critical thinking, creativity, and collaboration.
5. **Assessment and Feedback**: Implement regular assessments and feedback mechanisms to monitor student progress, identify areas for improvement, and adjust the curriculum accordingly.

**Incorporating Emerging Technologies:**

1. **Artificial Intelligence (AI) and Machine Learning (ML)**: Integrate AI and ML concepts, tools, and techniques into the curriculum, focusing on their applications and implications.
2. **Internet of Things (IoT)**: Explore the IoT's potential and challenges, including data analytics, security, and ethics.
3. **Data Science and Analytics**: Develop skills in data analysis, interpretation, and visualization, using real-world datasets and case studies.
4. **Cybersecurity**: Teach students about cybersecurity fundamentals, threats, and best practices, emphasizing the importance of online safety and digital citizenship.
5. **Virtual and Augmented Reality (VR/AR)**: Incorporate VR/AR technologies, exploring their potential in education, entertainment, and industry.

**Teacher Training and Support:**

1. **Professional Development**: Provide ongoing professional development opportunities for teachers to stay updated on the latest technologies and pedagogical methods.
2. **Mentorship and Coaching**: Offer mentorship and coaching programs to support teachers in implementing the new curriculum and addressing the evolving needs of their students.
3. **Teacher Communities**: Foster online and offline communities for teachers to share resources, best practices, and experiences, promoting collaboration and innovation.

**Continuous Evaluation and Improvement:**

1. **Regular Review and Revision**: Regularly review and revise the curriculum to ensure it remains relevant, effective, and aligned with the rapidly changing technological landscape.
2. **Feedback Mechanisms**: Establish feedback mechanisms to collect input from students, teachers, and industry experts, informing curriculum updates and improvements.
3. **Assessment and Evaluation**: Develop and implement effective assessment and evaluation methods to measure student learning outcomes, curriculum effectiveness, and teacher performance.

By following this framework, educators can create a balanced curriculum that develops critical thinking skills, masters specific knowledge, and prepares students for success in a rapidly changing technological landscape.

## For the next cell, we will use Ollama

Ollama runs a local web service that gives an OpenAI compatible endpoint,  
and runs models locally using high performance C++ code.

If you don't have Ollama, install it here by visiting https://ollama.com then pressing Download and following the instructions.

After it's installed, you should be able to visit here: http://localhost:11434 and see the message "Ollama is running"

You might need to restart Cursor (and maybe reboot). Then open a Terminal (control+\`) and run `ollama serve`

Useful Ollama commands (run these in the terminal, or with an exclamation mark in this notebook):

`ollama pull <model_name>` downloads a model locally  
`ollama ls` lists all the models you've downloaded  
`ollama rm <model_name>` deletes the specified model from your downloads

<table style="margin: 0; text-align: left; width:100%">
    <tr>
        <td style="width: 150px; height: 150px; vertical-align: middle;">
            <img src="../assets/stop.png" width="150" height="150" style="display: block;" />
        </td>
        <td>
            <h2 style="color:#ff7800;">Super important - ignore me at your peril!</h2>
            <span style="color:#ff7800;">The model called <b>llama3.3</b> is FAR too large for home computers - it's not intended for personal computing and will consume all your resources! Stick with the nicely sized <b>llama3.2</b> or <b>llama3.2:1b</b> and if you want larger, try llama3.1 or smaller variants of Qwen, Gemma, Phi or DeepSeek. See the <A href="https://ollama.com/models">the Ollama models page</a> for a full list of models and sizes.
            </span>
        </td>
    </tr>
</table>

In [ ]:
!ollama pull llama3.2

In [ ]:
ollama = OpenAI(base_url='http://localhost:11434/v1', api_key='ollama')
model_name = "llama3.2"

response = ollama.chat.completions.create(model=model_name, messages=messages)
answer = response.choices[0].message.content

display(Markdown(answer))
competitors.append(model_name)
answers.append(answer)

In [ ]:
# So where are we?

print(competitors)
print(answers)


['gpt-4o-mini', 'gemini-2.0-flash', 'llama-3.3-70b-versatile']
["Designing a new society from scratch is a complex task that requires careful consideration of ethical principles, governance structures, and potential cultural conflicts. Here’s a framework for such a society that prioritizes fairness, sustainability, and innovation:\n\n### Core Ethical Principles\n\n1. **Equity and Justice**: All individuals should have equal access to resources, opportunities, and decision-making processes. This includes addressing historical injustices and ensuring representation of marginalized groups.\n\n2. **Sustainability**: Societal development should meet the needs of the present without compromising the ability of future generations to meet their own needs. This involves prioritizing environmental stewardship and ecological balance.\n\n3. **Innovation and Adaptability**: Encourage creativity and innovation through open access to knowledge and technology. Encourage flexibility in social structure

In [ ]:
# It's nice to know how to use "zip"
for competitor, answer in zip(competitors, answers):
    print(f"Competitor: {competitor}\n\n{answer}")


Competitor: gpt-4o-mini

Designing a new society from scratch is a complex task that requires careful consideration of ethical principles, governance structures, and potential cultural conflicts. Here’s a framework for such a society that prioritizes fairness, sustainability, and innovation:

### Core Ethical Principles

1. **Equity and Justice**: All individuals should have equal access to resources, opportunities, and decision-making processes. This includes addressing historical injustices and ensuring representation of marginalized groups.

2. **Sustainability**: Societal development should meet the needs of the present without compromising the ability of future generations to meet their own needs. This involves prioritizing environmental stewardship and ecological balance.

3. **Innovation and Adaptability**: Encourage creativity and innovation through open access to knowledge and technology. Encourage flexibility in social structures to adapt to evolving needs and challenges.

4.

In [ ]:
# Let's bring this together - note the use of "enumerate"

together = ""
for index, answer in enumerate(answers):
    together += f"# Response from competitor {index+1}\n\n"
    together += answer + "\n\n"

In [ ]:
print(together)

# Response from competitor 1

Designing a new society from scratch is a complex task that requires careful consideration of ethical principles, governance structures, and potential cultural conflicts. Here’s a framework for such a society that prioritizes fairness, sustainability, and innovation:

### Core Ethical Principles

1. **Equity and Justice**: All individuals should have equal access to resources, opportunities, and decision-making processes. This includes addressing historical injustices and ensuring representation of marginalized groups.

2. **Sustainability**: Societal development should meet the needs of the present without compromising the ability of future generations to meet their own needs. This involves prioritizing environmental stewardship and ecological balance.

3. **Innovation and Adaptability**: Encourage creativity and innovation through open access to knowledge and technology. Encourage flexibility in social structures to adapt to evolving needs and challenges

In [ ]:
judge = f"""You are judging a competition between {len(competitors)} competitors.
Each model has been given this question:

{question}

Your job is to evaluate each response for clarity and strength of argument, and rank them in order of best to worst.
Respond with JSON, and only JSON, with the following format:
{{"results": ["best competitor number", "second best competitor number", "third best competitor number", ...]}}

Here are the responses from each competitor:

{together}

Now respond with the JSON with the ranked order of the competitors, nothing else. Do not include markdown formatting or code blocks."""


In [ ]:
print(judge)

You are judging a competition between 3 competitors.
Each model has been given this question:

If you were tasked with designing a new society from scratch, what core ethical principles and governance structures would you prioritize to ensure fairness, sustainability, and innovation, and how would you address potential conflicts that arise from differing cultural values?

Your job is to evaluate each response for clarity and strength of argument, and rank them in order of best to worst.
Respond with JSON, and only JSON, with the following format:
{"results": ["best competitor number", "second best competitor number", "third best competitor number", ...]}

Here are the responses from each competitor:

# Response from competitor 1

Designing a new society from scratch is a complex task that requires careful consideration of ethical principles, governance structures, and potential cultural conflicts. Here’s a framework for such a society that prioritizes fairness, sustainability, and inno

In [ ]:
judge_messages = [{"role": "user", "content": judge}]

In [ ]:
# Judgement time!

openai = OpenAI()
response = openai.chat.completions.create(
    model="o3-mini",
    messages=judge_messages,
)
results = response.choices[0].message.content
print(results)


{"results": ["2", "3", "1"]}


In [ ]:
# OK let's turn this into results!

results_dict = json.loads(results)
ranks = results_dict["results"]
for index, result in enumerate(ranks):
    competitor = competitors[int(result)-1]
    print(f"Rank {index+1}: {competitor}")

Rank 1: gemini-2.0-flash
Rank 2: llama-3.3-70b-versatile
Rank 3: gpt-4o-mini


<table style="margin: 0; text-align: left; width:100%">
    <tr>
        <td style="width: 150px; height: 150px; vertical-align: middle;">
            <img src="../assets/exercise.png" width="150" height="150" style="display: block;" />
        </td>
        <td>
            <h2 style="color:#ff7800;">Exercise</h2>
            <span style="color:#ff7800;">Which pattern(s) did this use? Try updating this to add another Agentic design pattern.
            </span>
        </td>
    </tr>
</table>

<table style="margin: 0; text-align: left; width:100%">
    <tr>
        <td style="width: 150px; height: 150px; vertical-align: middle;">
            <img src="../assets/business.png" width="150" height="150" style="display: block;" />
        </td>
        <td>
            <h2 style="color:#00bfff;">Commercial implications</h2>
            <span style="color:#00bfff;">These kinds of patterns - to send a task to multiple models, and evaluate results,
            are common where you need to improve the quality of your LLM response. This approach can be universally applied
            to business projects where accuracy is critical.
            </span>
        </td>
    </tr>
</table>

In [14]:
request = "You are given 5 ingredients egg, bread, cliantro, butter, chicken. You have come up with a unique dish that caters to taste buds of both Italian and Indian. Keep in mind that you can use only these ingredients maybe supplements like salt , pepper and oil are exceptionable . Give your response in a structured way like first the ingredients with the quantity and preparation and then at last serving."
messages = [{"role": "user", "content": request}]

In [15]:
model_name = "gpt-4o-mini"

response = openai.chat.completions.create(model=model_name, messages=messages)
recipe = response.choices[0].message.content

chain = []
models = []
models.append(model_name)
chain.append(recipe)

In [19]:

request = f"you are given a recipe: {recipe}, but it is not quite gourment , make it gourment and format the answer in the same way as the recipe given to you. "
messages = [{"role": "user", "content": request}]
model_name = "gemini-2.0-flash"

response = gemini.chat.completions.create(model=model_name, messages=messages)
answer = response.choices[0].message.content

In [20]:
chain.append(answer)

In [21]:
models.append(model_name)

In [22]:
request = f"you are given a gourment recipe: {answer}, but make sure it doesn't contain allergens like peanuts and diary products. "
messages = [{"role": "user", "content": request}]

model_name = "llama-3.3-70b-versatile"

response = groq.chat.completions.create(model=model_name, messages=messages)
answer_without_allergens = response.choices[0].message.content

models.append(model_name)
chain.append(answer_without_allergens)

In [25]:
together = ""
for index, chain in enumerate(chain):
    together += f"# Response from competitor {index+1}\n\n"
    together += chain + "\n\n"

In [ ]:
print(together)

In [29]:
display(Markdown(answer_without_allergens))

To modify the recipe to avoid common allergens like peanuts and dairy products, here are some adjustments you can make:

### Modified Dish Name: Cilantro Chicken Egg Bread Soufflé Bake with Saffron Aioli (Dairy-Free, Peanut-Free)

#### Ingredients:

*   **Chicken**: 350 grams, free-range chicken breast, cut into 1cm dice
*   **Eggs**: 4 large, organic, free-range (Ensure the eggs are from a peanut-free and dairy-free environment)
*   **Bread**: 4 slices, artisan sourdough or brioche made with non-dairy milk and without peanuts, crusts removed
*   **Cilantro**: 1/2 cup, finely chopped, plus extra sprigs for garnish
*   **Butter**: Replace with 3 tablespoons of dairy-free, peanut-free butter or margarine (such as vegan butter)
*   **Salt**: Fleur de sel, to taste
*   **Black Pepper**: Freshly cracked Tellicherry peppercorns, to taste
*   **Olive Oil**: 1 tablespoon, extra virgin, cold-pressed
*   **Shallots**: 1 small, finely minced
*   **Garlic**: 1 clove, minced
*   **Non-Dairy Cream**: 1/4 cup (such as almond milk, soy milk, or coconut cream)
*   **Dairy-Free Cheese**: 1/4 cup, finely grated (such as vegan Parmesan cheese or nutritional yeast)
*   **Saffron Aioli**: (Modified Recipe Follows)

#### Modified Saffron Aioli Ingredients:

*   **Mayonnaise**: 1/2 cup, homemade or high-quality store-bought dairy-free and peanut-free mayonnaise
*   **Saffron Threads**: Pinch (about 1/4 teaspoon)
*   **Garlic**: 1 clove, minced
*   **Lemon Juice**: 1 teaspoon, freshly squeezed
*   **Hot Sauce**: Few drops (such as Tabasco or your favorite, ensure it's dairy-free and peanut-free)
*   **Salt**: Pinch, to taste

#### Preparation:

1.  **Marinate Chicken**:
    *   In a bowl, combine the diced chicken with Fleur de sel, freshly cracked Tellicherry pepper, and 2 tablespoons of finely chopped cilantro. Add a drizzle of extra virgin olive oil. Let it marinate in the refrigerator for at least 30 minutes, or up to 2 hours.
2.  **Prepare Bread**:
    *   Lightly toast the artisan sourdough or brioche slices in a pan with 1 tablespoon of dairy-free butter or margarine until golden brown and crispy. Cut each slice into 1.5cm cubes.
3.  **Prepare Saffron Aioli**:
    *   In a small bowl, steep the saffron threads in 1 tablespoon of hot water for 5 minutes to release their color and flavor.
    *   In another bowl, combine the dairy-free mayonnaise, garlic, lemon juice, and hot sauce. Add the saffron and its soaking liquid. Mix well and season with a pinch of salt. Refrigerate until serving.
4.  **Cook Chicken**:
    *   In a skillet, melt 2 tablespoons of dairy-free butter or margarine with extra virgin olive oil over medium heat.
    *   Add the minced shallots and garlic and sauté until fragrant, about 1-2 minutes.
    *   Add the marinated chicken to the skillet and cook until it is lightly browned and cooked through, about 5-7 minutes. Stir occasionally to ensure even cooking. Remove from heat and set aside.
5.  **Prepare Egg Mixture**:
    *   In a bowl, crack the organic, free-range eggs and whisk them vigorously with a fork, creating a fluffy base. Add Fleur de sel, freshly cracked Tellicherry pepper, the remaining finely chopped cilantro, and non-dairy cream. Mix thoroughly.
6.  **Combine Ingredients**:
    *   Butter a small soufflé dish (approximately 6-inch diameter and 3 inches high) with dairy-free butter or margarine to prevent sticking.
    *   Layer the toasted bread cubes at the bottom of the prepared dish.
    *   Spread the cooked chicken evenly over the bread.
    *   Sprinkle the dairy-free cheese over the chicken.
    *   Pour the egg mixture evenly over the layered bread, chicken, and cheese, ensuring every piece is moistened. Gently press down on the bread to encourage absorption.
7.  **Bake**:
    *   Preheat your oven to 375°F (190°C).
    *   Bake in the preheated oven for 30-35 minutes, or until the soufflé is puffed up, golden brown, and the eggs are set. A slight jiggle in the center is acceptable.

#### Serving:

*   Once baked, allow the soufflé bake to cool slightly for a couple of minutes before serving. This allows it to set a little.
*   Garnish with fresh cilantro sprigs.
*   Serve immediately, with a generous dollop of modified saffron aioli on the side.

By making these adjustments, you can enjoy a delicious and allergy-friendly version of the Cilantro Chicken Egg Bread Soufflé Bake with Saffron Aioli.

In [30]:
display(Markdown(answer))

Okay, here's a gourmet version of the Cilantro Chicken Egg Bread Bake, elevating the ingredients and presentation while retaining the core concept.

### Dish Name: Cilantro Chicken Egg Bread Soufflé Bake with Saffron Aioli

#### Ingredients:

*   **Chicken**: 350 grams, free-range chicken breast, cut into 1cm dice
*   **Eggs**: 4 large, organic, free-range
*   **Bread**: 4 slices, artisan sourdough or brioche, crusts removed
*   **Cilantro**: 1/2 cup, finely chopped, plus extra sprigs for garnish
*   **Butter**: 3 tablespoons, unsalted, European-style
*   **Salt**: Fleur de sel, to taste
*   **Black Pepper**: Freshly cracked Tellicherry peppercorns, to taste
*   **Olive Oil**: 1 tablespoon, extra virgin, cold-pressed
*   **Shallots**: 1 small, finely minced
*   **Garlic**: 1 clove, minced
*   **Heavy Cream**: 1/4 cup
*   **Parmesan Cheese**: 1/4 cup, finely grated, aged at least 24 months
*   **Saffron Aioli**: (Recipe Follows)

#### Saffron Aioli Ingredients:

*   **Mayonnaise**: 1/2 cup, homemade or high-quality store-bought
*   **Saffron Threads**: Pinch (about 1/4 teaspoon)
*   **Garlic**: 1 clove, minced
*   **Lemon Juice**: 1 teaspoon, freshly squeezed
*   **Hot Sauce**: Few drops (such as Tabasco or your favorite)
*   **Salt**: Pinch, to taste

#### Preparation:

1.  **Marinate Chicken**:
    *   In a bowl, combine the diced chicken with Fleur de sel, freshly cracked Tellicherry pepper, and 2 tablespoons of finely chopped cilantro. Add a drizzle of extra virgin olive oil. Let it marinate in the refrigerator for at least 30 minutes, or up to 2 hours.
2.  **Prepare Bread**:
    *   Lightly toast the artisan sourdough or brioche slices in a pan with 1 tablespoon of butter until golden brown and crispy. Cut each slice into 1.5cm cubes.
3.  **Prepare Saffron Aioli**:
    *   In a small bowl, steep the saffron threads in 1 tablespoon of hot water for 5 minutes to release their color and flavor.
    *   In another bowl, combine the mayonnaise, garlic, lemon juice, and hot sauce. Add the saffron and its soaking liquid. Mix well and season with a pinch of salt. Refrigerate until serving.
4.  **Cook Chicken**:
    *   In a skillet, melt 2 tablespoons of unsalted European-style butter with extra virgin olive oil over medium heat.
    *   Add the minced shallots and garlic and sauté until fragrant, about 1-2 minutes.
    *   Add the marinated chicken to the skillet and cook until it is lightly browned and cooked through, about 5-7 minutes. Stir occasionally to ensure even cooking. Remove from heat and set aside.
5.  **Prepare Egg Mixture**:
    *   In a bowl, crack the organic, free-range eggs and whisk them vigorously with a fork, creating a fluffy base. Add Fleur de sel, freshly cracked Tellicherry pepper, the remaining finely chopped cilantro, and heavy cream. Mix thoroughly.
6.  **Combine Ingredients**:
    *   Butter a small soufflé dish (approximately 6-inch diameter and 3 inches high) to prevent sticking.
    *   Layer the toasted bread cubes at the bottom of the prepared dish.
    *   Spread the cooked chicken evenly over the bread.
    *   Sprinkle the grated Parmesan cheese over the chicken.
    *   Pour the egg mixture evenly over the layered bread, chicken, and cheese, ensuring every piece is moistened. Gently press down on the bread to encourage absorption.
7.  **Bake**:
    *   Preheat your oven to 375°F (190°C).
    *   Bake in the preheated oven for 30-35 minutes, or until the soufflé is puffed up, golden brown, and the eggs are set. A slight jiggle in the center is acceptable.

#### Serving:

*   Once baked, allow the soufflé bake to cool slightly for a couple of minutes before serving. This allows it to set a little.
*   Garnish with fresh cilantro sprigs.
*   Serve immediately, with a generous dollop of saffron aioli on the side.

Enjoy this elevated and refined take on a classic comfort dish. The saffron aioli adds a touch of luxury and complements the savory flavors of the cilantro chicken and egg bread soufflé beautifully.


In [31]:
display(Markdown(recipe))

### Dish Name: Cilantro Chicken Egg Bread Bake

#### Ingredients:
- **Chicken**: 300 grams, boneless and cut into small pieces
- **Eggs**: 4 large
- **Bread**: 4 slices (preferably whole grain or sourdough)
- **Cilantro**: 1/2 cup, finely chopped
- **Butter**: 2 tablespoons
- **Salt**: to taste
- **Black Pepper**: to taste
- **Oil**: 1 tablespoon (olive oil recommended)

#### Preparation:

1. **Marinate Chicken**:
   - In a bowl, combine the chicken pieces with salt, black pepper, and 2 tablespoons of chopped cilantro. Add a drizzle of olive oil and let it marinate for at least 15-20 minutes.

2. **Prepare Bread**:
   - Lightly toast the bread slices until they are crispy but not burnt. You can use a toaster or a skillet for this. Once toasted, cut each slice into quarters or cubes.

3. **Cook Chicken**:
   - In a skillet, heat the butter and a tablespoon of olive oil over medium heat.
   - Add the marinated chicken to the skillet and cook until the chicken is golden brown and fully cooked, about 7-10 minutes. Make sure to stir occasionally to ensure even cooking.

4. **Prepare Egg Mixture**:
   - In a bowl, crack the eggs and whisk them well. Add salt, pepper, and the remaining cilantro. Mix thoroughly.

5. **Combine Ingredients**:
   - In a baking dish, layer the toasted bread cubes at the bottom. 
   - Spread the cooked chicken evenly over the bread.
   - Pour the egg mixture over the layered bread and chicken, ensuring every piece is moistened.

6. **Bake**:
   - Preheat your oven to 350°F (175°C).
   - Bake in the preheated oven for about 25-30 minutes, or until the eggs are set and the top is golden brown.

#### Serving:
- Once baked, allow the dish to cool slightly before serving.
- Cut into squares or rectangles and garnish with additional cilantro on top.
- Serve hot, optionally with a side of yogurt or a simple salad for a fresh contrast. 

Enjoy this fusion dish that combines the culinary flavors of both Italian and Indian cuisine with a unique twist!